# Additional metrics with VMAF

#### Source: https://github.com/Netflix/vmaf

##### ADM, VIF, PSNR, SSIM and MS-SSIM

When using the filter `libvmaf` with ffmpeg to compute VMAF, other metrics such as MS-SSIM, SSIM, and PSNR can be computed.

## Example:

As the distorted video is in a different resolution, it is upscaled:

In [ ]:
!ffmpeg -hide_banner -y -i videos/dist/x264/bbb_640x360_CRF23.mp4 -vf scale=1920x1080 -c:v libx264 -crf 0 /tmp/dist.mp4

The following command computes the metrics using ffmpeg. In this example, the output is the file `results/libvmaf-metrics/bbb_metrics.json` that contains the metric values per frame. 

In [ ]:
! mkdir -p results/libvmaf-metrics && \
ffmpeg -hide_banner \
        -r 25 -i videos/ref/bbb_1920x1080.mp4 \
        -r 25 -i /tmp/dist.mp4 \
        -lavfi "[0:v]setpts=PTS-STARTPTS[reference]; \
         [1:v]setpts=PTS-STARTPTS[distorted]; \
         [distorted][reference]libvmaf=log_fmt=json:psnr=1:ssim=1:ms_ssim=1:log_fmt=json:\
         log_path=results/libvmaf-metrics/bbb_metrics.json:model='path=/vmaf/model/vmaf_float_v0.6.1.json'" -f null -

#### Graphical representation.

This section shows the graphical representation of the metrics obtained together with VMAF.

In [ ]:
import plotly.express as px
import pandas as pd
import json
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

# File with the metrics (adatp to your needs)
path_file='results/libvmaf-metrics/bbb_metrics.json'

with open(path_file) as f:
  data = json.load(f)

vmaf_frames=[]

for frame in data['frames']:
    vmaf_frames.append(frame['metrics'])
    
df=pd.DataFrame(vmaf_frames)
df.columns = map(str.upper, df.columns)

app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Video quality metrics integrated into libvmaf"),
    dcc.Graph(id='graph'),
    html.Label([
        "Select a metric:",
        dcc.Dropdown(
            id='metric-dropdown', clearable=False,
            value='ADM2', options=[
                {'label': c, 'value': c}
                for c in list(df.columns)
            ])
    ]),
])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("metric-dropdown", "value")]
)
def update_figure(metric):
    fig= px.line(df,x=df.index, y=metric,title="<b>"+str(metric)+' by frame'+"</b>",
    labels={"index": "Frame"}, template='simple_white',markers=True)
    fig.update_layout(title={'y':0.9,'x':0.5,'xanchor': 'center'})
    
    return fig

# Run app and display result inline in the notebook
app.run_server(host="0.0.0.0",mode='inline')